# Libs

In [2]:
#general 
import cv2
import numpy as np
%pylab inline

#gym
from gym import Env
from gym.spaces import Discrete, Box

#communication
import socket
import pickle
import struct

import time



Populating the interactive namespace from numpy and matplotlib


# Settings


In [35]:
ip = '192.168.0.202'


# Socket connection functions

In [36]:

#S is now a socket



def send(s, data):
    data= pickle.dumps(data, protocol=2) #RPI runs python 2, PC python 3!!
    s.sendall(struct.pack('>i',len(data)))
    s.sendall(data)


def recv(s):
    data = s.recv(4, socket.MSG_WAITALL)
    data_len = struct.unpack('>i', data)[0]
    data = s.recv(data_len, socket.MSG_WAITALL)
    return pickle.loads(data)
    

# OpenCV

In [37]:
# This is the first webcam connected to the computer
webcam = cv2.VideoCapture(0)

In [38]:
def getPos():
    retval, image = webcam.read()
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    blur = cv2.medianBlur(image_rgb,15)
    hsv = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)
    #color definition
    red_lower = np.array([80,0,245])
    red_upper = np.array([100,150,255])
    mask = cv2.inRange(hsv, red_lower, red_upper)
    mask.max()

    maxY = 480
    maxX = 600
    x, y, w, h = cv2.boundingRect(mask)

    centerX =  (x+(w/2)) -(maxX/2)
    centerY = (y+(h/2)) - (maxY/2)
    return centerX, centerY

# Built ENV

In [236]:
class moveLedEnv(Env):
    def __init__(self):
        
        # Actions we can take: up/left/right/down/stay
        self.action_space = Discrete(5)

        # Observations: gyroscope+accelerometer
        self.observation_space = Box(low=np.array([0,0]), high=np.array([8,8]), dtype=np.float32) #8 because sensehat 6 (gyr+acc) + openCV 2 (x+y)

        # Set start observation
        self.state = [0,0]

        self.reward = 0

        self.done = False

        self.distanceTolerance = 0.4


        
    def step(self, action):
        #runs every time we take a step in our environment

        self.done=False

        # Apply action
        # 0 stay
        # 1 left 
        # 2 right 
        # 3 up
        # 4 down

        def stay():
            self.state=self.state

        def left():
            if self.state[0]>0:
                self.state[0] -=1

        def right():
            if self.state[0]<7:
                self.state[0]+=1

        def up():
            if self.state[1]<7:
                self.state[1]+=1

        def down():
            if self.state[1]>0:
                self.state[1]-=1

        
        switcher={
            0: stay,
            1: left,
            2: right,
            3: up,
            4: down
        }
        
        switcher[action]()

        time.sleep(500) #wait a little to not being to fast
        x,y = getPos()
        time.sleep(500) #wait a little to not being to fast

        # Calculate reward
        self.reward = -(abs(x) + abs(y))/100 #x and y are distances from the center, the further, the higher -> the closer the less negative the reward
        

        

        info = {}

        s= socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.setsockopt(socket.IPPROTO_TCP, socket.TCP_NODELAY,1)
        s.connect((ip,9395))
        
        pos = self.state
        send(s,pos)

        if self.reward> -self.distanceTolerance:
            self.done = True

        # Return step information
        return self.state, self.reward, self.done, info



    def render(self):
        # Implement viz - not done
        pass
    
    def reset(self):
        s= socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.setsockopt(socket.IPPROTO_TCP, socket.TCP_NODELAY,1)
        s.connect((ip,9395))
        # Reset led
        self.reward=0
        self.state = [0,0]
        self.done = False
        send(s,self.state)
        return self.state

## Testing

In [237]:
env = moveLedEnv()


In [238]:
# 0 stay
# 1 left 
# 2 right 
# 3 up
# 4 down
env.step(4)

([0, 0], -0.245, True, {})

In [239]:
env.reset()

[0, 0]

In [240]:
action = 3
n_state, reward, done, info = env.step(action)
reward

-2.38

In [241]:
getPos()

(-98.5, -102.0)

In [242]:

episodes = 50
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score=reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:-0.31
Episode:2 Score:-0.31
Episode:3 Score:-0.315
Episode:4 Score:-0.31
Episode:5 Score:-0.315
Episode:6 Score:-0.245
Episode:7 Score:-0.31
Episode:8 Score:-0.315


KeyboardInterrupt: 

# MAIN

In [243]:
from stable_baselines3 import PPO, DQN


In [244]:
model = DQN("MlpPolicy", env, verbose=1)


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [252]:
model.learn(total_timesteps=10000)


----------------------------------
| rollout/            |          |
|    ep_len_mean      | 75       |
|    ep_rew_mean      | -144     |
|    exploration rate | 0.715    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 11       |
|    time_elapsed     | 25       |
|    total timesteps  | 300      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 54       |
|    ep_rew_mean      | -99      |
|    exploration rate | 0.59     |
| time/               |          |
|    episodes         | 8        |
|    fps              | 11       |
|    time_elapsed     | 37       |
|    total timesteps  | 432      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 57.2     |
|    ep_rew_mean      | -107     |
|    exploration rate | 0.348    |
| time/               |          |
|    episodes       

In [253]:
model.save('myModel3')

In [262]:
env.reset()

[0, 0]

In [269]:
env.step(3)

([1, 4], -1.605, False, {})

In [272]:
obs = env.reset()
while True:
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)

    if done:
      obs = env.reset()

KeyboardInterrupt: 